# Set-up

Deliverables:

1-You must create a simple web API to predict a user's BMI in real-time. You can also use webcam input. The goal is to use one of the pre-trained image models (e.g. VGG Face), fine-tune with the provided data, and deploy via jupyter notebook, streamlit, flask or any other simple restful api's.

2-10 pages of the write-up about your implementation.

3-10 mins presentation or live demo in the final lecture.

Our goal is to beat the performance metrics provided in the paper.

In [1]:
import os
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive/')
path_gdrive = '/content/drive/MyDrive/Colab Datasets/ML/BMI'
os.chdir(path_gdrive)
print(os.getcwd())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Datasets/ML/BMI


In [ ]:
# read csv file
data_path = '/content/drive/MyDrive/Colab Datasets/ML/BMI/all_data.csv'
bmi = pd.read_csv(data_path)

In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [4]:
import tensorflow as tf
import keras

In [5]:
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.12.0
Keras version: 2.12.0


In [6]:
! pip install keras_vggface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Prepare Keras

In [2]:
# !python --version

Python 3.10.11


In [8]:
# https://stackoverflow.com/questions/68862735/keras-vggface-no-module-named-keras-engine-topology
#! pip install git+https://github.com/rcmalli/keras-vggface.git
! pip install keras_applications --no-deps

filename = "/usr/local/lib/python3.10/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.7 MB/s eta 0:00:00


20951

## Extact Features

1. Read images with Kreas API: https://machinelearningmastery.com/how-to-load-convert-and-save-images-with-the-keras-api/

In [8]:
bmi.head()

,name,path,bmi,gender,is_training,imgae_type
0,img_0.bmp,Data/Images/img_0.bmp,34.207396,Male,1,bmp
1,img_1.bmp,Data/Images/img_1.bmp,26.453720,Male,1,bmp
2,img_10.bmp,Data/Images/img_10.bmp,38.732782,Male,1,bmp
3,img_100.bmp,Data/Images/img_100.bmp,29.834105,Male,1,bmp
4,img_1000.bmp,Data/Images/img_1000.bmp,42.240827,Male,1,bmp


In [9]:
train = bmi[bmi.is_training == 1]
val = bmi[bmi.is_training != 1]

In [ ]:
import tensorflow as tf
from keras_vggface.vggface import VGGFace
from keras.models import Model

# Initialize a model for feature extraction
vggface = VGGFace()

In [17]:
# Match outputs to fc6 features
model = Model(inputs=vggface.input, outputs=vggface.get_layer('fc6').output)

# Define layer name for feature extraction
layer_name = 'fc6'

In [18]:
# example of loading an image with the Keras API
from keras.preprocessing.image import *

def extract_features(img_path):
  if os.path.exists(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0) # Add one dimension (channel)
    x = utils.preprocess_input(x, version=1)
    features = model.predict(x)
    return features.flatten( )
  else:
    return None

Note:

1. Transfer Learning. https://medium.com/@leosimmons/estimating-body-mass-index-from-face-images-using-keras-and-transfer-learning-de25e1bc0212